1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고,
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500, # 하나의 chunk 가 가질수 있는 토큰 수 
    chunk_overlap=200, # 겹치는 부분 -> 유사도 검색을 할 때 우리가 원하는 문서를 가지고 오는 확률을 높이기 위함함
)

loader = Docx2txtLoader("../tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [7]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# index = pc.Index(index_name)

database = PineconeVectorStore.from_documents(documents=document_list, embedding=embedding, index_name=index_name)


In [8]:
query = '연봉 5000만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query=query, k=3)

In [9]:
retrived_docs

[Document(id='6fa4aff7-3f93-4a80-8465-15a905e132b8', metadata={'source': '../tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [11]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

In [12]:
ai_message = llm.invoke(prompt)

'연봉 5,000만원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 과세 표준과 세율을 고려해야 합니다. 2021년 기준으로 한국의 소득세율은 다음과 같습니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 ~ 4,600만원 이하: 15%\n3. 4,600만원 초과 ~ 8,800만원 이하: 24%\n4. 8,800만원 초과 ~ 1억5,000만원 이하: 35%\n5. 1억5,000만원 초과 ~ 3억원 이하: 38%\n6. 3억원 초과 ~ 5억원 이하: 40%\n7. 5억원 초과: 42%\n\n단, 연말정산 시 세액공제, 인적공제, 특별공제 등 여러 공제 항목이 적용되므로 실제 납부해야 하는 세금은 다를 수 있습니다.\n\n연봉 5,000만원의 소득세를 계산할 때 기본적인 세율에 따른 계산 방법은 다음과 같습니다.\n\n1. 1,200만원까지: 1,200만원 * 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원: (4,600만원 - 1,200만원) * 15% = 510만원\n3. 4,600만원 초과 ~ 5,000만원: (5,000만원 - 4,600만원) * 24% = 96만원\n\n따라서, 총 소득세는 72만원 + 510만원 + 96만원 = 678만원입니다.\n\n위 계산은 기본적인 세율을 기준으로 한 것일 뿐, 실제 세액은 개인의 상황에 따라 달라질 수 있습니다. 정확한 세액은 연말정산 또는 세무 전문가의 상담을 통해 확인하실 수 있습니다.'


In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/data/anaconda3/envs/langchain/lib/python3.13/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query = '연봉 5000만원인 직장인의 소득세는 얼마인가요?'

In [16]:
ai_message = qa_chain({"query":query})

/tmp/ipykernel_3123234/980961933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query":query})


In [17]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5,000만원인 직장인의 소득세는 복잡한 세법에 따라 계산되며, 소득 세율과 공제를 고려해야 합니다. 일반적으로 연소득에 따라 세율이 다르며, 추정세액을 계산할 수 있는 다양한 온라인 계산기를 활용할 수 있습니다. 정확한 금액을 알고 싶다면 세무사와 상담하는 것이 좋습니다.'}